# **Abrir la siguiente URL para abrir el notebook en colab , un ide online , las librerias pesan aproximadamente 6GB por lo que se recomienda correr el codigo ahi. **

In [ ]:
https://colab.research.google.com/drive/12aw93WMdyCygvjrpsBluV4v0UoQsY3eC?usp=sharing

# **Ejecutar la siguiente celda ( puede demorar alrededor de 1 minutos en colab)**

In [ ]:
!pip install pinecone-client sentence-transformers ipywidgets deep_translator

from sentence_transformers import SentenceTransformer
from deep_translator import GoogleTranslator
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

from pinecone import Pinecone
import pandas as pd
import ast


pc = Pinecone(api_key="19316538-e983-4d75-9868-da68f4439da9")
index = pc.Index("amazon")


import ipywidgets as widgets
from IPython.display import display, HTML
import random


# Función para realizar la búsqueda y traducción
def buscar_y_traducir(query, idioma_origen, idioma_destino):
    # Traducir la consulta al inglés
    translator_to_en = GoogleTranslator(source=idioma_origen, target='en')
    query_en = translator_to_en.translate(query)

    # Vectorizar la consulta
    query_embedding = model.encode(query_en).tolist()

    # Realizar la búsqueda en Pinecone
    results = index.query(
        namespace="ns1",
        vector=query_embedding,
        top_k=12,
        include_values=False,
        include_metadata=True,
        filter={}
    )

    # Traducir los resultados al idioma destino
    translator_from_en = GoogleTranslator(source='en', target=idioma_destino)
    productos_traducidos = []
    for match in results['matches']:
        titulo_original = match['metadata'].get('title', '')
        titulo_traducido = translator_from_en.translate(titulo_original)
        productos_traducidos.append({
            'nombre': titulo_traducido,
            'precio': match['metadata'].get('price', 0),
            'valoracion': match['metadata'].get('stars', 0),
            'imagen': match['metadata'].get('imgUrl', ''),
            'link': match['metadata'].get('productURL', '')
        })

    return productos_traducidos


# Función de búsqueda actualizada
def realizar_busqueda(b):
    query = busqueda.value
    if not query:
        return

    with resultados:
        resultados.clear_output()
        display(loading)
        idioma_seleccionado = idioma.value
        productos = buscar_y_traducir(query, idioma_seleccionado, idioma_seleccionado)
        resultados.clear_output()
        display(mostrar_productos(productos))
        display(HTML(f"<p>Resultados de búsqueda para: <strong>{query}</strong></p>"))

        # Estilos CSS para el header similar a Amazon



# Ejecutar para cargar el diseño

In [ ]:
estilos = """
<style>
    body {
        background-color: #f0f4f8;
        font-family: 'Helvetica Neue', Arial, sans-serif;
    }
    .header {
        background-color: #232f3e;
        padding: 10px 20px;
        display: flex;
        align-items: center;
        justify-content: space-between;
        color: white;
        font-size: 14px;
    }
    .logo {
        font-size: 24px;
        font-weight: bold;
    }
    .search-bar {
    flex: 1 !important;
    margin: 0 20px !important;
    padding: 12px 25px !important;
    border-radius: 50px !important;
    border: 1px solid #ddd !important;
    font-size: 15px !important;
    background-color: #f5f5f5 !important;
    color: #555 !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 8px 15px rgba(0, 0, 0, 0.1) !important;
    font-family: 'Helvetica Neue', sans-serif !important;
}

.search-bar:focus {
    outline: none !important;
    border-color: #3498db !important;
    box-shadow: 0 0 15px rgba(52, 152, 219, 0.7) !important;
    background-color: #fff !important;
}

.search-bar::placeholder {
    color: #aaa !important;
    font-style: italic !important;
}


    .search-button {
        background-color: #febd69;
        border: 1px solid #f90;
        font-size: 16px;
        cursor: pointer;
        border-radius: 4px;
        color: black;
        font-weight: bold;
    }
    .search-button:hover {
        background-color: #f90;
    }
    .language-selector {
        margin-left: 5px;
        width: 8%;
    }
    .language-selector select {
        border-radius: 4px;
        border: 1px solid #ccc;
        background-color: #3498db; /* Nuevo color azul para el botón de idioma */
        color: white;
        cursor: pointer;
        padding: 5px 10px;
    }
    .tienda-container {
        max-width: 1200px;
        margin: 0 auto;
        padding: 20px;
    }
    .productos-grid {
        display: grid;
        grid-template-columns: repeat(auto-fill, minmax(250px, 1fr));
        gap: 25px;
        padding: 30px 0;
    }
    .producto-card {
        background-color: white;
        border-radius: 12px;
        box-shadow: 0 8px 16px rgba(0,0,0,0.1);
        overflow: hidden;
        transition: transform 0.3s ease, box-shadow 0.3s ease;
    }
    .producto-card:hover {
        transform: translateY(-5px);
        box-shadow: 0 12px 20px rgba(0,0,0,0.15);
    }
    .producto-imagen {
        width: 100%;
        height: 200px;
        object-fit: cover;
    }
    .producto-info {
        padding: 20px;
    }
    .producto-nombre {
        font-size: 18px;
        font-weight: bold;
        margin-bottom: 10px;
        color: #333;
    }
    .producto-precio {
        font-size: 16px;
        color: #e74c3c;
        font-weight: bold;
    }
    .producto-valoracion {
        color: #f39c12;
        margin-top: 5px;
    }
    .producto-link {
        display: inline-block;
        margin-top: 10px;
        padding: 8px 15px;
        background-color: #3498db;
        color: white;
        text-decoration: none;
        border-radius: 5px;
        transition: background-color 0.3s ease;
    }
    .producto-link:hover {
        background-color: #2980b9;
    }
    .loading-container {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 50px;
        margin-top: 20px;
    }
</style>
"""


# Función para mostrar productos
def mostrar_productos(productos):
    html = estilos + "<div class='tienda-container'>"
    html += "<div class='productos-grid'>"
    for producto in productos:
        html += f"""
        <div class='producto-card'>
            <img src='{producto["imagen"]}' class='producto-imagen' alt='{producto["nombre"]}'>
            <div class='producto-info'>
                <div class='producto-nombre'>{producto["nombre"]}</div>
                <div class='producto-precio'>${producto["precio"]:.2f}</div>
                <div class='producto-valoracion'>{'★' * int(producto["valoracion"])}</div>
                <a href='{producto["link"]}' target='_blank' class='producto-link'>Ver en Amazon</a>
            </div>
        </div>
        """
    html += "</div></div>"
    return HTML(html)


    # Widgets actualizados
busqueda = widgets.Text(placeholder="Busca productos...", layout=widgets.Layout(flex='1'))
boton_buscar = widgets.Button(description="Buscar", layout=widgets.Layout(width='auto'))
idioma = widgets.Dropdown(
    options=[('Español', 'es'), ('English', 'en'), ('Français', 'fr')],
    value='es',
    layout=widgets.Layout(width='auto')
)
resultados = widgets.Output()
loading = widgets.HTML('''
<div class="loading-container">
    <script src="https://unpkg.com/@dotlottie/player-component@latest/dist/dotlottie-player.mjs" type="module"></script>
    <dotlottie-player src="https://lottie.host/d067c7ba-cfc3-4d82-b5ab-6046b51f3a14/pVWK5frgrG.json" background="transparent" speed="1" style="width: 100px; height: 100px;" loop autoplay></dotlottie-player>
</div>
''')




# Conectar la función de búsqueda al botón y al evento de Enter
boton_buscar.on_click(realizar_busqueda)
busqueda.on_submit(realizar_busqueda)

# Mostrar la interfaz actualizada
header_html = """
<div class="header">
    <div class="logo"><img src="https://sopyfive.com/external/amazon.png" alt="Amazon Logo" style="height: 30px; margin-top: 10px"></div>
    <div id="busqueda-container" style="flex: 1; display: flex;"></div>
    <div id="idioma-container" class="language-selector"></div>
</div>
"""

display(HTML(estilos + header_html))
display(widgets.HBox([busqueda, boton_buscar], layout=widgets.Layout(display='flex', flex='1')))
display(widgets.HBox([idioma], layout=widgets.Layout(display='flex', justify_content='flex-end')))
display(resultados)

# Aplicar estilos personalizados a los widgets
display(HTML("""
<script>
    var busquedaElement = document.querySelector('.widget-text input');
    busquedaElement.classList.add('search-bar');
    var botonBuscarElement = document.querySelector('.widget-button');
    botonBuscarElement.classList.add('search-button');
    var idiomaElement = document.querySelector('.widget-dropdown select');
    idiomaElement.classList.add('language-selector');

    var busquedaContainer = document.getElementById('busqueda-container');
    busquedaContainer.appendChild(busquedaElement.parentElement);
    busquedaContainer.appendChild(botonBuscarElement);

    var idiomaContainer = document.getElementById('idioma-container');
    idiomaContainer.appendChild(idiomaElement.parentElement);
</script>
"""))

# **Código que se utilizó en su momento para procesar datos y montar la BD en pinecone**

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cargar el archivo CSV
file_path_cat = '/content/drive/MyDrive/datasets/amazon_products/amazon_categories.csv'
df_cat = pd.read_csv(file_path_cat)
file_path = '/content/drive/MyDrive/datasets/amazon_products/amazon_products.csv'
df = pd.read_csv(file_path)

In [ ]:
# Combinar las tablas en base a 'category_id'
df_combined = pd.merge(final_df, categories, left_on='category_id', right_on='id', how='left')

# Combinar las columnas para vectorizar ambas
df_combined['combined_text'] = df_combined['title'] + ' ' + df_combined['category_name']

# Seleccionar la columna combinada
combined_texts = df_combined['combined_text'].tolist()

In [ ]:
# Generar embeddings para los textos combinados con barra de progreso y tamaño de lote
embeddings = model.encode(combined_texts, show_progress_bar=True, batch_size=512)

# Añadir los embeddings al DataFrame
final_df['embeddings'] = embeddings.tolist()

output_file_path = '/content/drive/MyDrive/datasets/amazon_products/amazon-embeddings.csv'
df_part.to_csv(output_file_path, index=False)


print(f"Archivo guardado en: {output_file_path}")

In [ ]:
# Subir la bd a pinecone por lotes

# Define el tamaño del fragmento
chunk_size = 500

for chunk in df_part:
    data = []
    for _, row in chunk.iterrows():
        try:
            embeddings = ast.literal_eval(row['embeddings'])

            data.append({
                'id': str(row['asin']),
                'values': embeddings,
                'metadata': {
                    'title': str(row['title']),
                    'imgUrl': str(row['imgUrl']),
                    'productURL': str(row['productURL']),
                    'stars': float(row['stars']),
                    'price': float(row['price']),
                    'category_id': str(row['category_id']),
                }
            })
        except (ValueError, SyntaxError) as e:
            print(f"Error al procesar la fila: {row['asin']}. Error: {e}")

    if data:
        try:
            index.upsert(vectors=data, namespace='ns1')
            print(f"Fragmento de {len(data)} vectores subido exitosamente.")
            vectors_count += len(data)
        except Exception as e:
            print(f"Error al subir el fragmento: {e}")
    else:
        print("No hay nuevos datos para subir en este fragmento.")

    print(f"Total de vectores subidos: {vectors_count}")

print("Proceso completado.")